<a href="https://colab.research.google.com/github/motorlearner/neuromatch/blob/main/laquitaine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# imports
import requests
import pandas as pd
import numpy as np

In [54]:
# @title Read Data
# ------------------------------------------------------------------------------

# fetch data
url = "https://github.com/steevelaquitaine/projInference/raw/gh-pages/data/csv/data01_direction4priors.csv"
try:
  RequestAPI = requests.get(url)
except requests.ConnectionError:
  print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
else:
  if RequestAPI.status_code != requests.codes.ok:
    print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
  else:
    with open("data01_direction4priors.csv", "wb") as fid:
      fid.write(RequestAPI.content)

# read data
data = pd.read_csv("data01_direction4priors.csv")

In [49]:
# @title Helpers: Process Data
# ---------------------------------------------------------------------------- #


# COORDINATE TRANSFORMS ------------------------------------------------------ #

def cart2pol(x:np.array, y:np.array):
  """
  Convert cartesian `(x,y)` to polar `(deg,mag)`.

  Args:
      `x`: scalar or array of x-coordinates
      `y`: scalar or array of y-coordinates

  Returns:
      Tuple of `(deg,mag)`, where `deg` is in the interval
      [0,360) counterclockwise from the positive x axis.
  """
  # checks
  # compute
  deg = (np.degrees(np.arctan2(y,x)) + 360) % 360
  mag = np.hypot(x,y)
  return deg,mag


def cart2deg(x:np.array, y:np.array):
  return cart2pol(x,y)[0]


def cart2mag(x:np.array, y:np.array):
  return cart2pol(x,y)[1]


def pol2cart(deg: np.array, mag: np.array):
  """
  Convert polar `(deg,mag)` to cartesian `(x,y)`.

  Args:
      `deg`: scalar or array of angles in degrees [0,360)
      `mag`: scalar or array of magnitudes

  Returns:
      Tuple of `(x, y)` coordinates.
  """
  # Convert degrees to radians
  rad = np.radians(deg)
  # Compute x and y
  x = mag * np.cos(rad)
  y = mag * np.sin(rad)
  return x, y

# CIRCULAR DISTANCE ---------------------------------------------------------- #

def circdiff(angle:np.array, reference:np.array):
  """
  Compute the signed minimal circular distance from `angle` to `reference`.

  Args:
      `angle`: scalar or array of angles in degrees [0, 360)
      `reference`: scalar reference angle in degrees [0, 360)

  Returns:
      Signed circular distance in degrees (-180, 180].
  """
  return ((angle - reference + 180) % 360) - 180

In [89]:
df = data.copy()
# stimulus relative to prior
df['motion_direction_rel'] = circdiff(df.motion_direction, df.prior_mean)
# polar estimate
df['estimate_deg'], df['estimate_mag'] = cart2pol(df.estimate_x, df.estimate_y)
# error (deg)
df['error'] = circdiff(df.estimate_deg, df.motion_direction)
# error (deg) +ve if towards prior_mean (even if overshoots)
df['prioratt'] = np.where(df.motion_direction_rel *  (e := df.error) < 0, np.abs(e), -np.abs(e))
df['prioratt_norm'] = np.where((mdr := df.motion_direction_rel) != 0, df.error / mdr, np.nan)

Index(['trial_index', 'trial_time', 'response_arrow_start_angle',
       'motion_direction', 'motion_coherence', 'estimate_x', 'estimate_y',
       'reaction_time', 'raw_response_time', 'prior_std', 'prior_mean',
       'subject_id', 'experiment_name', 'experiment_id', 'session_id',
       'run_id', 'motion_direction_rel', 'estimate_deg', 'estimate_mag',
       'error', 'prioratt', 'prioratt_norm'],
      dtype='object')

In [94]:
def plot_runs(subject_id):

  dat = df[df.subject_id == subject_id]

  session_ids =  dat.session_id.unique()






In [95]:
plot_runs(1)

array([1, 2, 3, 4, 5, 6, 7, 8])